In [1]:
!pip install paddlehub==2.1.1

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 KB 4.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.8 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 4.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


In [2]:
import pandas as pd
train = pd.read_csv('./ner_work/train_data_public.csv')
test = pd.read_csv('./ner_work/test_public.csv')
train

,id,text,BIO_anno,class
0,0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0
1,1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1
2,2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0
3,3,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
4,4,利率不错，可以撸,B-COMMENTS_N I-COMMENTS_N B-COMMENTS_ADJ I-COM...,1
...,...,...,...,...
7523,7523,我鼎级拒了,O O O B-COMMENTS_ADJ O,2
7524,7524,一打一个准，准胜，看激活信用卡时那协议，全是对银行有利的,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
7525,7525,招行分期白80k,B-BANK I-BANK B-PRODUCT I-PRODUCT I-PRODUCT O O O,2
7526,7526,5万，额度还行吧没毕业哦,O O O B-COMMENTS_N I-COMMENTS_N O O O O O O O,2


In [15]:
import paddle
import paddlehub as hub
from paddlehub.datasets.base_nlp_dataset import BaseNLPDataset
from typing import Dict, List, Optional, Union, Tuple

from paddlehub.datasets.base_nlp_dataset import TextClassificationDataset
from paddlehub.text.bert_tokenizer import BertTokenizer
from paddlehub.text.tokenizer import CustomTokenizer

In [16]:
# Step2，使用PaddleHub
model = hub.Module(name='ernie_tiny', version='2.0.1', task='seq-cls', num_classes=3)

[2022-04-28 12:33:27,246] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-tiny/ernie_tiny.pdparams


In [17]:
# Step1，自定义数据集
class MyDataset(TextClassificationDataset):
    """DemoDataset"""
    def __init__(self, tokenizer: Union[BertTokenizer, CustomTokenizer], max_seq_len: int = 128, mode: str = 'train'):
        # 数据集存放位置
        base_path = "/home/aistudio/ner_work"

        if mode == 'train':
            data_file = 'ernie_train.tsv'
        elif mode == 'test':
            data_file = 'ernie_test.tsv'
        else:
            data_file = 'ernie_dev.tsv'
        super().__init__(
            base_path=base_path,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            data_file=data_file,
            label_list=["0", "1", "2"],
            is_file_with_header=True)


In [18]:
# 数据集设置
train_dataset = MyDataset(tokenizer=model.get_tokenizer(), max_seq_len=512, mode='train')
test_dataset = MyDataset(tokenizer=model.get_tokenizer(), max_seq_len=512, mode='test')


[2022-04-28 12:33:39,628] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/vocab.txt
[2022-04-28 12:33:39,631] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/spm_cased_simp_sampled.model
[2022-04-28 12:33:39,634] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/dict.wordseg.pickle
[2022-04-28 12:33:46,625] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/vocab.txt
[2022-04-28 12:33:46,628] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/spm_cased_simp_sampled.model
[2022-04-28 12:33:46,630] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-tiny/dict.wordseg.pickle


In [19]:
# Step3，设置优化器，运行配置
optimizer = paddle.optimizer.AdamW(learning_rate=5e-5, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='./ernie_checkpoint', use_gpu=True)

trainer.train(
    train_dataset,
    epochs=3,
    batch_size=32,
    save_interval=1,
)


[2022-04-28 12:33:53,114] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...
[2022-04-28 12:33:56,948] [   TRAIN] - Epoch=1/3, Step=10/236 loss=0.6346 acc=0.7688 lr=0.000050 step/sec=2.61 | ETA 00:04:31
[2022-04-28 12:34:00,621] [   TRAIN] - Epoch=1/3, Step=20/236 loss=0.4818 acc=0.8781 lr=0.000050 step/sec=2.72 | ETA 00:04:25
[2022-04-28 12:34:04,304] [   TRAIN] - Epoch=1/3, Step=30/236 loss=0.4262 acc=0.8625 lr=0.000050 step/sec=2.72 | ETA 00:04:24
[2022-04-28 12:34:07,972] [   TRAIN] - Epoch=1/3, Step=40/236 loss=0.4581 acc=0.8313 lr=0.000050 step/sec=2.73 | ETA 00:04:22
[2022-04-28 12:34:11,653] [   TRAIN] - Epoch=1/3, Step=50/236 loss=0.3793 acc=0.8688 lr=0.000050 step/sec=2.72 | ETA 00:04:22
[2022-04-28 12:34:15,347] [   TRAIN] - Epoch=1/3, Step=60/236 loss=0.4010 acc=0.8500 lr=0.000050 step/sec=2.71 | ETA 00:04:22
[2022-04-28 12:34:19,024] [   TRAIN] - Epoch=1/3, Step=70/236 loss=0.3169 acc=0.8875 lr=0.000050 step/sec=2.72 | ETA 00:04:22
[2022-04-28 12:34:2

In [20]:
# Step4，预测
label_map = {0: 0, 1: 1, 2: 2}
model = hub.Module(
    name='ernie_tiny',
    version='2.0.1',
    task='seq-cls',
    load_checkpoint='./ernie_checkpoint/epoch_3/model.pdparams', label_map=label_map)
model


[2022-04-28 12:38:24,251] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-tiny/ernie_tiny.pdparams
[2022-04-28 12:38:27,747] [    INFO] - Loaded parameters from /home/aistudio/ernie_checkpoint/epoch_3/model.pdparams


ErnieTiny(
  (model): ErnieForSequenceClassification(
    (ernie): ErnieModel(
      (embeddings): ErnieEmbeddings(
        (word_embeddings): Embedding(50006, 1024, padding_idx=0, sparse=False)
        (position_embeddings): Embedding(600, 1024, sparse=False)
        (token_type_embeddings): Embedding(2, 1024, sparse=False)
        (layer_norm): LayerNorm(normalized_shape=[1024], epsilon=1e-05)
        (dropout): Dropout(p=0.1, axis=None, mode=upscale_in_train)
      )
      (encoder): TransformerEncoder(
        (layers): LayerList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiHeadAttention(
              (q_proj): Linear(in_features=1024, out_features=1024, dtype=float32)
              (k_proj): Linear(in_features=1024, out_features=1024, dtype=float32)
              (v_proj): Linear(in_features=1024, out_features=1024, dtype=float32)
              (out_proj): Linear(in_features=1024, out_features=1024, dtype=float32)
            )
            (linear1): Lin

In [21]:
[[x] for x in test['text'].values]

[['共享一个额度，没啥必要，四个卡不要年费吗？你这种人头，银行最喜欢，广发是出了名的风控严，套现就给你封...'],
 ['炸了，就2000.浦发没那么好心，草'],
 ['挂了电话自己打过去分期提额可以少分一点的'],
 ['比如你首卡10k，二卡也10k，信报上显示邮政总共给你的授信额度是20k'],
 ['3000吗，浦发总是这样'],
 ['直接抛答案，没有最低只有更低，没什么可比性。'],
 ['那个网点激活的，能顺便办储蓄卡吗'],
 ['3000，那要养到啥是才有3万额度啊'],
 ['30周年庆是随便开吗，额度也随便批么'],
 ['哦，我搞错了?我的是中信的'],
 ['下卡了吗？招行的下的很快'],
 ['一般都是同时申请鼎至鼎雅，鼎至下了当然最好，不然还有鼎雅保底。'],
 ["没有希望?和我一样，恒丰美食卡，浦东发展评论很慢?别人'"],
 ['您太厉害了，我估计是碰到了不靠谱的理财经理'],
 ['你打招行的电话问问'],
 ['请问楼主被拒，禁推人员是单位不符合还是个人征信不符合'],
 ['全部3K那就尴尬了'],
 ['不回访电话核实嘛？我当时办交通银行的就电话核实了'],
 ['没有快递信息，一般是分批发送。?我昨天下午到的，很快'],
 ['这什么操作看不懂，有鼎致了，竟然鼎雅批不下来？'],
 ['正好卡15天，不过不是15个工作日啊'],
 ['线下激活问你网上申请填的一些资料，如果你答不上来'],
 ['五月底还给了五万消费专用备用金，不过现在点开没有了，就是不提额，临额都没有'],
 ['十万存在银行才有多少利息，存在华为5G开放性概念基金十万明天不动，利息88，一年是多少，十年又是多少，要是自己懂得买，赚的更多'],
 ['我吐了……第十三行，一千也太低了，不是说好五千买菜嘛？'],
 ['直接上去，卡是可以办，就看额度了'],
 ['确实也很过分，但是投诉也没必要吧'],
 ['感觉没啥必要弄吧，额度太低了'],
 ['哪有啥电话26个小时了，都是骗人的吧'],
 ['不知道啊，没经历过刚刷就来电这种情况。邮储这么猛？这以后咋刷'],
 ['你是什么时候出的邀请，我24000多白条都没有邀请'],
 ['总感觉这个版区的版主挺不容易的，，，好不容易培育出一个银行热门，，，就要闹独立分出去'],
 ['可以以卡办卡，额

In [25]:
result = model.predict([[x] for x in test['text'].values], max_seq_len=512, batch_size=32, use_gpu=True)
result

[2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,


In [26]:
test_result = pd.read_csv('./predict2-Ernie-aistudio.csv')
test_result['class'] = result
test_result

,id,BIO_anno,class
0,0,O O O O B-COMMENTS_N I-COMMENTS_N O O O O O O ...,2
1,1,O O O O O O O O O B-BANK I-BANK O O O B-COMMEN...,0
2,2,O O O O O O O O O B-PRODUCT I-PRODUCT B-COMMEN...,2
3,3,O O O B-PRODUCT I-PRODUCT O O O O B-PRODUCT I-...,2
4,4,O O O O O O B-BANK I-BANK O O O O,0
...,...,...,...
2878,2878,O O O O O O O O O O O O B-PRODUCT I-PRODUCT O O,2
2879,2879,O O O B-COMMENTS_ADJ I-COMMENTS_ADJ O O O O B-...,2
2880,2880,O O O O O O O B-PRODUCT I-PRODUCT O O O O O O ...,2
2881,2881,O B-PRODUCT I-PRODUCT I-PRODUCT B-COMMENTS_N I...,0


In [28]:
test_result.to_csv('./predict4-class-paddlehub-aistuido.csv', index=False)